# Traveling Salesman Problem

## Model Initialization

In [3]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp

# import pandas package
import pandas as pd

# initialize model object
mymodel = glp.Solver('Traveling Salesman', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

In [4]:
# import cost matrix as data frame
cost = pd.DataFrame(
    [[0.0, 5.0, 15.0, 10.0, 0.0], # cost from node 1
     [5.0, 0.0, 12.0, 20.0, 5.0],  # node 2
     [15.0, 12.0, 0.0, 6.0, 15.0],  # node 3
     [10.0, 20.0, 6.0, 0.0, 10.0], # node 4
     [0.0, 5.0, 15.0, 10.0, 0.0]]) # dummy node

# coerce data type = integer / numeric?
# Probably need a way to create the dummny data from a regular cost matrix

# row = starting node, column = ending node (i to j)

In [5]:
# model parameters
N = len(cost.index) # number of nodes in data frame (including the dummy node)
M = N + 1000.0 # arbitrarily large number

## Decision Variables

In [6]:
# create arc variable
use_arc = pd.DataFrame(index = range(cost.shape[0]), columns = range(cost.shape[1]))

for i in range(N):
    for j in range(N):
        use_arc[j][i] = mymodel.IntVar(0, 1, str(i+1) + "." + str(j+1))
use_arc

,0,1,2,3,4
0,1.1,1.2,1.3,1.4,1.5
1,2.1,2.2,2.3,2.4,2.5
2,3.1,3.2,3.3,3.4,3.5
3,4.1,4.2,4.3,4.4,4.5
4,5.1,5.2,5.3,5.4,5.5


In [7]:
# create position variable (lambda / u)
pos = list(range(N))
for i in range(N):
    pos[i] = mymodel.IntVar(0, N, "p_" + str(i+1))
pos

[p_1, p_2, p_3, p_4, p_5]

## Objective

In [8]:
# create objective function
shortest_route = mymodel.Objective()
shortest_route.SetMinimization()
for i in range(N):
    for j in range(N):
        shortest_route.SetCoefficient(use_arc[j][i], cost[j][i])

## Constraints

### Arc constraints

In [9]:
# first node - 1.1 + 1.2 + 1.3 + 1.4 + 1.5 = 1
#orig_node = mymodel.Constraint(1, 1)
#for i in range(N):
#    orig_node.SetCoefficient(use_arc.loc[0][i], 1)

# last node - 1.5 + 2.5 + 3.5 + 4.5 + 5.5 = 1
#last_node = mymodel.Constraint(1, 1)
#for j in range(N):
#    last_node.SetCoefficient(use_arc.loc[j][N-1], 1)

# Can't go from and to the same node - 1.1 + 2.2 + 3.3 + 4.4 + 5.5 = 1
#same_node = mymodel.Constraint(0, 0)
#for i in range(N):
#    same_node.SetCoefficient(use_arc.loc[i][i], 1)
    
# Can't go from first to dummy (1.5)
#first_to_last = mymodel.Constraint(0, 0)
#first_to_last.SetCoefficient(use_arc[N-1][0], 1)

# Can't go from dummy to first (5.1)
#last_to_first = mymodel.Constraint(0, 0)
#last_to_first.SetCoefficient(use_arc[0][N-1], 1)

In [10]:
# Balance - Out
# 2.1 + 2.2 + 2.3 + 2.4 + 2.5 = 1

# Balance - In
# 1.2 + 2.2 + 3.2 + 4.2 + 5.2 = 1

# Flow Out

Flow_out = pd.Series(index = range(N))

for i in range(0, N-1): 
    Flow_out[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        Flow_out[i].SetCoefficient(use_arc[j][i], 1)
        
# Flow In

Flow_in = pd.Series(index = range(N))

for i in range(1, N):
    Flow_in[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        Flow_in[i].SetCoefficient(use_arc[i][j], 1)

### Position Constraints

In [11]:
# position 1 = 1
position_1 = mymodel.Constraint(1, 1)
position_1.SetCoefficient(pos[0], 1)

# position N = N
position_N = mymodel.Constraint(1, 1)
position_N.SetCoefficient(pos[N-1], N)

# MTZ Constraint

# u[i] - u[j] + (n-1)*x[i][j] <= (n-2)

MTZ = pd.DataFrame(index = range(N), columns = range(N))

for i in range(1, N):
    for j in range(1, N):
        MTZ[j][i] = mymodel.Constraint(-mymodel.infinity(), N-2)
        MTZ[j][i].SetCoefficient(use_arc[j][i], N-1)
        MTZ[j][i].SetCoefficient(pos[i], 1)
        MTZ[j][i].SetCoefficient(pos[j], -1)

In [12]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % shortest_route.Value())

# Display optimal solution
for i in range(N):
    for j in range(N):
        print('From ', i + 1, ' to ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
        
#        if i == N-1 or i == j:
#            continue
#        elif j == N-1:
 #           j = 1
 #           print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
 #       else:
  #          print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '') 

Solution Status = 2
Number of variables = 30
Number of constraints = 26
Optimal Solution:
Optimal Value = 0.00
From 1 to 1: 0.0
From 1 to 2: 0.0
From 1 to 3: 0.0
From 1 to 4: 0.0
From 1 to 5: 0.0
From 2 to 1: 0.0
From 2 to 2: 0.0
From 2 to 3: 0.0
From 2 to 4: 0.0
From 2 to 5: 0.0
From 3 to 1: 0.0
From 3 to 2: 0.0
From 3 to 3: 0.0
From 3 to 4: 0.0
From 3 to 5: 0.0
From 4 to 1: 0.0
From 4 to 2: 0.0
From 4 to 3: 0.0
From 4 to 4: 0.0
From 4 to 5: 0.0
From 5 to 1: 0.0
From 5 to 2: 0.0
From 5 to 3: 0.0
From 5 to 4: 0.0
From 5 to 5: 0.0
